In [ ]:
import pandas as pd

In [ ]:
times = []
with open('R1_time.txt') as f:
    for line in f:
        times.append(line.split('\t')[2].rstrip('\n'))
times

In [ ]:
times[0]

In [ ]:
array = times[0].split(':')#コロンを除去

In [ ]:
float(array[0]) * 3600 + float(array[1])*60+float(array[2])#時間表示を秒に変換する式,

In [ ]:
R1 = pd.read_csv('R1.txt', names=['time', 'ch1', 'ch2', 'ch3', 'ch4'])

In [ ]:
R1['time'][0]

In [ ]:
R1

In [ ]:
len(R1)

In [ ]:
R1['move'] = 0

In [ ]:
R1

In [ ]:
print(type(times[0]))

In [ ]:
import datetime

In [ ]:
int(times[0])
print(times[0].seconds)